In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import warnings
warnings.simplefilter(action='ignore')

Vaihe 1<br>
Iris-darasetin voi ladata suoraan sklearn.datasets-kirjastosta komennolla load_iris(), ja se laitetaan X ja y muuttujiin, jossa X on ominaisuusmatriisi ja y on target-muuttuja.<br>

Vaihe 2<br>
Data jaetaan training ja validaatiodataan.<br>

Vaihe 3<br>
Muunnetaan Numpy-taulukot tensoriksi: X_train ja X_val muutetaan float32 ja y_train ja y_val muutetaan long(kokonaislukuihin, koska luokkamerkinnät ovat 0, 1 ja 2)<br>

Vaihe 4<br>
Luodaan TensorDataset-objektit: yhdistetään X_train ja y_train ja X_val ja y_val tensorit yhdeksi TensorDataset-objektiksi.<br>

Vaihe 5<br>
Luodaan DataLoader-objektit: Se mahdollistaa datan lataamisen pienissä erissä ja satunnaisessa järjestyksessä.

In [3]:
#1
iris = load_iris()
X = iris.data
y = iris.target

#2
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#3
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

#4
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

#5
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

Seuraavassa koodisolussa määritellään yksinkertainen neuroverkkomalli PyTorchilla. Se perii nn.Module-luokan, joka on pytorchin perusluokka malleille.<br>
- self.hidden: Lineaarinen piilokerros, joka ottaa 4 sisääntuloa (koska datasetillä on 4 ominaisuutta) ja tuottaa 10 ulostuloa.
- self.output: Lineaarinen ulostulokerros, joka ottaa 10 sisääntuloa ja tuottaa 3 ulostuloa (koska datasetillä on 3 luokkaa).<br>

forward(): Määrittelee laskentapolun syötteestä ulostuloon eli mitä mallin tulee tehdä, kun sille syötetään dataa.
- x = torch.relu: Laskee piilotetun kerroksen arvot,
- x = self.output: Laskee ulostulokerroksen arvot ilman aktivointia. CrossEntropyLoss-luokka laskee softmaxin automaattisesti.

In [5]:
class neuroverkko(nn.Module):
    def __init__(self):
        super(neuroverkko, self).__init__()
        self.hidden = nn.Linear(4, 10) 
        self.output = nn.Linear(10, 3)

    def forward(self, x):
        x = torch.relu(self.hidden(x))
        x = self.output(x)
        return x

model = neuroverkko()

Seuraavassa solussa toteutetaan neuroverkon koulutus- ja validointisilmukka käyttäen PyTorchia.<br>
Ensiksi määritellään tappiofunktio ja optimointialgoritmi. Koulutusloopissa käydään läpi mallin koulutus 20 kertaa (epookia).<br>
- model.train(): Asettaa mallin koulutustilaan.
- correct_train = 0 ja total_train = 0: Asetetaan laskurit koulutustarkkuuden mittaamiseksi.
- for batch_X, batch_y in train_loader: Käydään läpi koulutusdata pienissä erissä, jonka jälkeen lasketaan koulutustarkkuus.

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
for epoch in range(20):
    model.train()
    correct_train = 0
    total_train = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        _, predicted_train = torch.max(outputs, 1)
        total_train += batch_y.size(0)
        correct_train += (predicted_train == batch_y).sum().item()
    train_accuracy = 100 * correct_train / total_train

    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            val_outputs = model(batch_X)
            _, predicted_val = torch.max(val_outputs, 1)
            total_val += batch_y.size(0)
            correct_val += (predicted_val == batch_y).sum().item()
    val_accuracy = 100 * correct_val / total_val

    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1, Loss: 1.0917, Train Accuracy: 39.17%, Validation Accuracy: 70.00%
Epoch 2, Loss: 0.9974, Train Accuracy: 65.83%, Validation Accuracy: 70.00%
Epoch 3, Loss: 1.0242, Train Accuracy: 65.83%, Validation Accuracy: 70.00%
Epoch 4, Loss: 0.8637, Train Accuracy: 65.83%, Validation Accuracy: 70.00%
Epoch 5, Loss: 0.7514, Train Accuracy: 65.83%, Validation Accuracy: 70.00%
Epoch 6, Loss: 0.6796, Train Accuracy: 66.67%, Validation Accuracy: 70.00%
Epoch 7, Loss: 0.5668, Train Accuracy: 70.00%, Validation Accuracy: 70.00%
Epoch 8, Loss: 0.7530, Train Accuracy: 70.00%, Validation Accuracy: 70.00%
Epoch 9, Loss: 0.5983, Train Accuracy: 69.17%, Validation Accuracy: 70.00%
Epoch 10, Loss: 0.7891, Train Accuracy: 76.67%, Validation Accuracy: 83.33%
Epoch 11, Loss: 0.6067, Train Accuracy: 84.17%, Validation Accuracy: 83.33%
Epoch 12, Loss: 0.4327, Train Accuracy: 89.17%, Validation Accuracy: 86.67%
Epoch 13, Loss: 0.4579, Train Accuracy: 90.00%, Validation Accuracy: 86.67%
Epoch 14, Loss: 0.567